目标：分析球员是否能铲球成功

# 1. 导入数据 <a class="anchor"  id="1"></a>

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import scipy.stats as stats
import numpy as np
from scipy.stats import linregress
from scipy.stats import norm
import plotly.figure_factory as ff
import plotly.io as pio

In [2]:
# 读取 games.csv 文件
games_data = pd.read_csv('../../Dataset/games.csv')

# 读取 players.csv 文件
players_data = pd.read_csv('../../Dataset/players.csv')

# 读取 plays.csv 文件
plays_data = pd.read_csv('../../Dataset/plays.csv')

# 读取 tackles.csv 文件
tackles_data = pd.read_csv('../../Dataset/tackles.csv')

建立一个函数用于统计表格相关列的值信息

In [3]:
def data_type_color(val):
    if "int" in val or "float" in val:
        color = '#88CCEE'  # 数值型数据颜色
    elif "object" in val:
        color = '#7D7D7D'  # 文本型数据颜色
    else:
        color = '#CC6677'  # 其他数据类型颜色
    return f'background-color: {color}'

def generate_summary_table(data):
    summary_table = pd.DataFrame(columns=['Column', 'Data Type', 'Missing Values', 'Missing %', 'Unique Values', 'Min', 'Max', 'Mean', 'Median'])
    
    for column in data.columns:
        data_type = str(data[column].dtype)
        
        missing_values = data[column].isnull().sum()
        missing_percentage = (missing_values / len(data)) * 100
        
        if data[column].dtype == 'object':
            min_value, max_value, mean_value, median_value = '', '', '', ''
        else:
            min_value = data[column].min()
            max_value = data[column].max()
            mean_value = data[column].mean()
            median_value = data[column].median()
        
        unique_values = data[column].nunique()
        
        summary_table = summary_table.append({
            'Column': column,
            'Data Type': data_type,
            'Missing Values': missing_values,
            'Missing %': f'{missing_percentage:.2f}%',
            'Unique Values': unique_values,
            'Min': min_value,
            'Max': max_value,
            'Mean': mean_value,
            'Median': median_value
        }, ignore_index=True)

    # 根据数据类型应用颜色
    def data_type_color(val):
        if "int" in val or "float" in val:
            color = '#88CCEE'  # 数值型数据颜色
        elif "object" in val:
            color = '#7D7D7D'  # 文本型数据颜色
        else:
            color = '#CC6677'  # 其他数据类型颜色
        return f'background-color: {color}'

    # 应用样式
    styled_table = summary_table.style.set_properties(**{
                    'text-align': 'center',
                    'font-size': '12pt',
                    'color': 'white',
                    'border': '1px solid gray'
                }).applymap(lambda x: 'background-color: #556677' if isinstance(x, (int, float)) else 'background-color: #334455').applymap(lambda x: 'background-color: #aa3333' if '%' in str(x) and float(x.strip('%')) > 20 else '').applymap(data_type_color, subset=['Data Type']).set_table_styles([{
                    'selector': 'th',
                    'props': [('background-color', '#445566'), ('color', 'white')]
                }])

    print("表格信息统计:")
    display(styled_table)
    
    return summary_table


1. 比赛数据games.csv分析

In [4]:
games_data.head()

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore
0,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31
1,2022091100,2022,1,09/11/2022,13:00:00,ATL,NO,26,27
2,2022091101,2022,1,09/11/2022,13:00:00,CAR,CLE,24,26
3,2022091102,2022,1,09/11/2022,13:00:00,CHI,SF,19,10
4,2022091103,2022,1,09/11/2022,13:00:00,CIN,PIT,20,23


生成表格信息统计图

In [5]:
# 调用函数并打印生成的表格
games_data_summary = generate_summary_table(games_data)

表格信息统计:


,Column,Data Type,Missing Values,Missing %,Unique Values,Min,Max,Mean,Median
0,gameId,int64,0,0.00%,136,2022090800,2022110700,2022098922.117647,2022100902.500000
1,season,int64,0,0.00%,1,2022,2022,2022.000000,2022.000000
2,week,int64,0,0.00%,9,1,9,4.845588,5.000000
3,gameDate,object,0,0.00%,27,,,,
4,gameTimeEastern,object,0,0.00%,8,,,,
5,homeTeamAbbr,object,0,0.00%,32,,,,
6,visitorTeamAbbr,object,0,0.00%,32,,,,
7,homeFinalScore,int64,0,0.00%,38,3,49,22.669118,22.500000
8,visitorFinalScore,int64,0,0.00%,35,0,48,20.948529,20.000000


2. 球员数据players.csv分析

In [6]:
players_data.head()

,nflId,height,weight,birthDate,collegeName,position,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan


生成表格信息统计图

In [7]:
# 调用函数并打印生成的表格
players_data_summary = generate_summary_table(players_data)

表格信息统计:


,Column,Data Type,Missing Values,Missing %,Unique Values,Min,Max,Mean,Median
0,nflId,int64,0,0.00%,1683,25511,55241,48221.702317,47872.000000
1,height,object,0,0.00%,16,,,,
2,weight,int64,0,0.00%,179,153,380,245.724302,236.000000
3,birthDate,object,479,28.46%,985,,,,
4,collegeName,object,0,0.00%,226,,,,
5,position,object,0,0.00%,19,,,,
6,displayName,object,0,0.00%,1672,,,,


3. 每场比赛数据plays.csv分析

In [8]:
plays_data.head()

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,...,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,...,0.976785,0.023215,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN
1,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,...,0.160485,0.839515,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,...,0.756661,0.243339,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN
3,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,...,0.620552,0.379448,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN
4,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,...,0.836290,0.163710,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN


生成表格信息统计图

In [9]:
# 调用函数并打印生成的表格
plays_data_summary = generate_summary_table(plays_data)

表格信息统计:


,Column,Data Type,Missing Values,Missing %,Unique Values,Min,Max,Mean,Median
0,gameId,int64,0,0.00%,136,2022090800,2022110700,2022098953.855598,2022100903.000000
1,playId,int64,0,0.00%,3974,54,5096,1986.603476,1990.500000
2,ballCarrierId,int64,0,0.00%,480,25511,55158,48072.271664,47789.000000
3,ballCarrierDisplayName,object,0,0.00%,480,,,,
4,playDescription,object,0,0.00%,12486,,,,
5,quarter,int64,0,0.00%,5,1,5,2.550136,3.000000
6,down,int64,0,0.00%,4,1,4,1.727054,2.000000
7,yardsToGo,int64,0,0.00%,32,1,38,8.469085,10.000000
8,possessionTeam,object,0,0.00%,32,,,,
9,defensiveTeam,object,0,0.00%,32,,,,


4. 铲球数据tackles.csv分析

In [10]:
tackles_data.head()

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800,101,42816,1,0,0,0
1,2022090800,393,46232,1,0,0,0
2,2022090800,486,40166,1,0,0,0
3,2022090800,646,47939,1,0,0,0
4,2022090800,818,40107,1,0,0,0


生成表格信息统计图

In [11]:
# 调用函数并打印生成的表格
tackles_data_summary = generate_summary_table(tackles_data)

表格信息统计:


,Column,Data Type,Missing Values,Missing %,Unique Values,Min,Max,Mean,Median
0,gameId,int64,0,0.00%,136,2022090800,2022110700,2022098971.441123,2022100903.000000
1,playId,int64,0,0.00%,3943,54,5096,1982.974578,1991.000000
2,nflId,int64,0,0.00%,800,33131,55241,47602.719442,46669.000000
3,tackle,int64,0,0.00%,2,0,1,0.569207,1.000000
4,assist,int64,0,0.00%,2,0,1,0.315276,0.000000
5,forcedFumble,int64,0,0.00%,2,0,1,0.005681,0.000000
6,pff_missedTackle,int64,0,0.00%,2,0,1,0.119936,0.000000


# 2. 可视化数据 <a class="anchor"  id="2"></a>

1. games.csv

首先对该表格内容进行可视化

由之前信息可得此表中没有出现数据缺失的情况，因此可以直接处理

得分分布 - 直方图展示主队和客队得分的分布情况

In [12]:
# 调用函数并打印生成的表格
games_data_summary = generate_summary_table(games_data)

表格信息统计:


,Column,Data Type,Missing Values,Missing %,Unique Values,Min,Max,Mean,Median
0,gameId,int64,0,0.00%,136,2022090800,2022110700,2022098922.117647,2022100902.500000
1,season,int64,0,0.00%,1,2022,2022,2022.000000,2022.000000
2,week,int64,0,0.00%,9,1,9,4.845588,5.000000
3,gameDate,object,0,0.00%,27,,,,
4,gameTimeEastern,object,0,0.00%,8,,,,
5,homeTeamAbbr,object,0,0.00%,32,,,,
6,visitorTeamAbbr,object,0,0.00%,32,,,,
7,homeFinalScore,int64,0,0.00%,38,3,49,22.669118,22.500000
8,visitorFinalScore,int64,0,0.00%,35,0,48,20.948529,20.000000


In [13]:
# 创建直方图
fig = px.histogram(games_data, x=["homeFinalScore", "visitorFinalScore"], nbins=20, barmode="overlay",
                   color_discrete_map={"homeFinalScore": "blue", "visitorFinalScore": "red"},
                   labels={"value": "Score", "variable": "Team"}, title="Score Distribution")

# 自定义颜色和透明度
fig.update_traces(opacity=0.7)

# 添加动画效果
fig.update_layout(transition_duration=500)

# 自定义字体样式
fig.update_layout(
    font=dict(family="Arial", size=14, color="black"),  # 自定义字体样式
    title=dict(font=dict(size=20)),  # 自定义标题样式
    xaxis_title=dict(font=dict(size=16)),  # 自定义X轴标题样式
    yaxis_title=dict(font=dict(size=16)),  # 自定义Y轴标题样式
)

# 显示图表
fig.show()

得分中间高两边底，比较符合正态分布，因此进一步分析，绘制出
* <b>正态概率图</b> - 数据分布应紧密地沿着代表正态分布的对角线分布。

In [14]:
# 绘制正态概率图
def create_qq_plot(scores, team_name, color):
    scores_sorted = np.sort(scores)
    scores_norm = stats.norm.ppf((np.arange(len(scores)) + 1) / (len(scores) + 1))

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=scores_norm, y=scores_sorted, mode='markers', 
                             name=f'{team_name} Scores', marker=dict(color=color, opacity=0.7)))

    fig.add_trace(go.Scatter(x=[min(scores_norm), max(scores_norm)], 
                             y=[min(scores_sorted), max(scores_sorted)], 
                             mode='lines', name='Theoretical Normal Distribution',
                             line=dict(color='grey', dash='dash')))

    fig.update_layout(title=f'{team_name} Score Normal Probability Plot',
                      xaxis_title='Theoretical Quantiles',
                      yaxis_title='Ordered Values',
                      template='plotly_white')

    return fig

fig_home = create_qq_plot(games_data['homeFinalScore'], 'Home Team', 'blue')
fig_home.show()

fig_visitor = create_qq_plot(games_data['visitorFinalScore'], 'Visitor Team', 'red')
fig_visitor.show()

通过正态概率图可以得出主客队的得分数据在很大程度上接近正态分布，尤其是在数据的中间区域。但是，数据的尾部可能有轻微的偏差。

赛季中的队伍表现 - 折线图显示每支球队在赛季不同周次的得分情况

In [15]:
# 将主客队得分合并到一个 DataFrame 中
home_scores = games_data[['season', 'week', 'homeTeamAbbr', 'homeFinalScore']].rename(
    columns={'homeTeamAbbr': 'Team', 'homeFinalScore': 'Score'})
visitor_scores = games_data[['season', 'week', 'visitorTeamAbbr', 'visitorFinalScore']].rename(
    columns={'visitorTeamAbbr': 'Team', 'visitorFinalScore': 'Score'})

# 合并主客队数据
team_scores = pd.concat([home_scores, visitor_scores])

# 根据赛季、周次和队伍分组，并计算得分总和
team_scores = team_scores.groupby(['season', 'week', 'Team']).agg({'Score': 'sum'}).reset_index()

# 获取所有队伍的列表
all_teams = team_scores['Team'].unique()

# 使用Plotly Express创建交互式折线图
fig = px.line(team_scores, x='week', y='Score', color='Team', 
              title='Team Performance Across Different Weeks',
              labels={'week': 'Week', 'Score': 'Score'})

# 设置布局
fig.update_layout(
    legend=dict(orientation='h', yanchor='bottom', y=-0.5, xanchor='center', x=0.5),
    margin=dict(l=20, r=20, t=40, b=20),
    plot_bgcolor='white',
    xaxis=dict(gridcolor='lightgray', title='Week'),
    yaxis=dict(gridcolor='lightgray', title='Score'),
    font=dict(family='Arial', size=12),
    hovermode='x'
)

# 减少所有线条的透明度
fig.for_each_trace(lambda trace: trace.update(opacity=0.7))

# 显示图表
fig.show()

可以看到队伍数目众多，难以观察，我们绘制处于上升趋势的队伍的得分折线图与处于下降趋势的队伍的得分折线图分别查看，由此可以判断队伍在整个赛季当中是进步还是退步。

还要绘制出得分变化较大的队伍的折线图，找出那些发挥不稳定的队伍

通过得分趋势的回归斜率判断队伍处于上升趋势还是下降趋势

In [16]:
# 根据赛季、周次和队伍分组，并计算得分总和
team_scores = team_scores.groupby(['season', 'week', 'Team'])['Score'].sum().reset_index()

# 计算每个队伍的回归斜率
slopes = {}
for team in team_scores['Team'].unique():
    team_data = team_scores[team_scores['Team'] == team]
    slope, _, _, _, _ = linregress(team_data['week'], team_data['Score'])
    slopes[team] = slope

# 对斜率进行排序并选取上升和下降趋势最强的五个队伍
ascending_teams = sorted(slopes, key=slopes.get, reverse=True)[:5]
descending_teams = sorted(slopes, key=slopes.get)[:5]

# 从 team_scores 中选取上升和下降趋势队伍的得分数据
ascending_scores = team_scores[team_scores['Team'].isin(ascending_teams)]
descending_scores = team_scores[team_scores['Team'].isin(descending_teams)]

上升趋势最强的5个队

In [17]:
# 绘制上升趋势的队伍得分
fig_ascend = px.line(ascending_scores, x='week', y='Score', color='Team',
                     title='Top 5 Teams with Ascending Score Trends',
                     labels={'week': 'Week', 'Score': 'Score'},
                     width=1000, height=600)
fig_ascend.show()
print(ascending_teams)

['DAL', 'CHI', 'SEA', 'SF', 'CIN']


可以看到'DAL', 'CHI', 'SEA', 'SF', 'CIN'这五个队在该赛季越战越勇，得分上升趋势最明显

下降趋势最强的5个队

In [18]:
# 绘制下降趋势的队伍得分
fig_descend = px.line(descending_scores, x='week', y='Score', color='Team',
                      title='Top 5 Teams with Descending Score Trends',
                      labels={'week': 'Week', 'Score': 'Score'},
                      width=1000, height=600)
fig_descend.show()
print(descending_teams)

['DET', 'BUF', 'PIT', 'KC', 'BAL']


可以看到'DET', 'BUF', 'PIT', 'KC', 'BAL'这五个队在该赛季越挫越败，得分下降趋势最明显

得分波动最大的五个队

In [19]:
# 计算每个队伍的得分波动性（标准偏差）
score_volatility = team_scores.groupby('Team')['Score'].std()

# 选取得分波动性最高的前5个队伍
top_5_unstable_teams = score_volatility.nlargest(5).index.tolist()

# 从team_scores中筛选出这5个队伍的数据
unstable_teams_scores = team_scores[team_scores['Team'].isin(top_5_unstable_teams)]

# 使用Plotly Express绘制折线图
import plotly.express as px

fig = px.line(unstable_teams_scores, x='week', y='Score', color='Team',
              title='Top 5 Unstable Teams Score Trends',
              labels={'week': 'Week', 'Score': 'Score'})

fig.show()

# 返回所选队伍的列表供参考
top_5_unstable_teams

['DET', 'DAL', 'SEA', 'LV', 'KC']

而'DET', 'DAL', 'SEA', 'LV', 'KC'这五个队得分波动最大，他们的表现具有很强的不稳定性

赛季中比赛结果统计 - 饼图展示每个赛季主队和客队的胜利次数和比赛结果

In [20]:
# 创建新列，表示主队和客队的胜利情况
games_data['home_win'] = games_data['homeFinalScore'] > games_data['visitorFinalScore']
games_data['visitor_win'] = games_data['homeFinalScore'] < games_data['visitorFinalScore']

# 统计每个赛季中主队和客队的胜利次数
season_results = games_data.groupby(['season']).agg({
    'home_win': 'sum',
    'visitor_win': 'sum'
}).reset_index()

# 重塑数据以符合饼图格式
season_results_melted = season_results.melt(id_vars='season', var_name='Result', value_name='Wins')

# 显示饼图
fig = px.pie(season_results_melted, values='Wins', names='Result', title='Season Results: Home Team vs Visitor Team Wins',
             hover_data=['Wins'], labels={'Result': 'Game Result'})
fig.show()

看来主场确实有优势，胜率高达54.1%。

进一步分析不同球队在主客场作战时是否有明显差异。

In [21]:
# 计算每个队伍的主场胜率
home_win_data = games_data.assign(HomeWin=games_data['homeFinalScore'] > games_data['visitorFinalScore'])
win_count = home_win_data.groupby('homeTeamAbbr')['HomeWin'].sum()
game_count = home_win_data['homeTeamAbbr'].value_counts()
win_rate = win_count / game_count

# 按胜率排序
sorted_teams = win_rate.sort_values(ascending=False).index

# 创建4x8的子图布局
fig = make_subplots(rows=4, cols=8, subplot_titles=sorted_teams, 
                    specs=[[{'type': 'pie'} for _ in range(8)] for _ in range(4)])

# 对每个队伍绘制饼图
for i, team in enumerate(sorted_teams):
    row = i // 8 + 1
    col = i % 8 + 1
    # 在饼图上显示标签和百分比
    fig.add_trace(go.Pie(labels=['Wins', 'Losses'], values=[win_count[team], game_count[team] - win_count[team]],
                         name=team, hoverinfo='label+percent', textinfo='label+percent'), row=row, col=col)

# 更新布局
fig.update_layout(height=600, width=1200, title_text="Home Win Rates for Different Teams", showlegend=False)

# 显示图表
fig.show()

In [22]:
# 按胜率排序队伍
sorted_teams = win_rate.sort_values(ascending=False).index

# 获取前五名和后五名队伍的名称
top_5_teams = sorted_teams[:5]
bottom_5_teams = sorted_teams[-5:]

# 打印队伍名称
print("主场胜率前五名队名:", top_5_teams)
print("主场败率前五名队名:", bottom_5_teams)

主场胜率前五名队名: Index(['MIN', 'BUF', 'PHI', 'DAL', 'MIA'], dtype='object')
主场败率前五名队名: Index(['CAR', 'NO', 'PIT', 'ARI', 'HOU'], dtype='object')


可以得出这五个队的主场优势极大'MIN', 'BUF', 'PHI', 'DAL', 'MIA'

这五个队几乎没有主场优势'CAR', 'NO', 'PIT', 'ARI', 'HOU'

2. players.csv

首先对该表格内容进行公制化转换，转换成我们可以处理的数据

身高：米  体重：千克  出生日期：转化为年龄

In [23]:
# 转换身高为厘米
players_data['height'] = players_data['height'].apply(lambda x: int(x.split('-')[0]) * 30.48 + int(x.split('-')[1]) * 2.54)

# 转换体重为公斤
players_data['weight'] = players_data['weight'] * 0.453592

# 计算年龄
players_data['birthDate'] = 2022 - pd.to_datetime(players_data['birthDate']).dt.year

# 输出转换后的结果
print(players_data)

      nflId  height      weight  birthDate       collegeName position  \
0     25511  193.04  102.058200       45.0          Michigan       QB   
1     29550  193.04  148.778176       40.0          Arkansas        T   
2     29851  187.96  102.058200       39.0        California       QB   
3     30842  198.12  121.109064       38.0              UCLA       TE   
4     33084  193.04   98.429464       37.0    Boston College       QB   
...     ...     ...         ...        ...               ...      ...   
1678  55200  198.12  120.655472        NaN           Indiana       DT   
1679  55212  182.88  104.326160        NaN        Iowa State      ILB   
1680  55239  187.96  136.077600        NaN      Pennsylvania       DT   
1681  55240  185.42   83.914520        NaN           Buffalo       CB   
1682  55241  187.96  127.005760        NaN  Coastal Carolina       DT   

           displayName  
0            Tom Brady  
1         Jason Peters  
2        Aaron Rodgers  
3       Marcedes Lewis 

接下来我们进行可视化分析

In [24]:
# 创建直方图统计身高每一厘米的频率
height_counts = players_data['height'].value_counts().sort_index().reset_index()
height_counts.columns = ['Height (cm)', 'Frequency']

fig = px.bar(height_counts, x='Height (cm)', y='Frequency', title='Height Distribution')
fig.update_layout(
    xaxis_title='Height (cm)',
    yaxis_title='Frequency',
    template='plotly_white'  # 使用明亮的主题
)

# 添加动画效果
fig.update_traces(marker_color='darkblue', opacity=0.7)
fig.update_layout(transition_duration=500)  # 设置动画持续时间

fig.show()

得分中间高两边底，比较符合正态分布，因此进一步分析，绘制出
* <b>正态概率图</b> - 数据分布应紧密地沿着代表正态分布的对角线分布。

In [25]:
fig_home = create_qq_plot(players_data['height'], 'Players Height', 'blue')
fig_home.show()

身高数据分布比较紧密地沿着代表正态分布的对角线分布，具有正态性

In [26]:
# 创建直方图统计体重
fig_weight = px.histogram(players_data, x='weight', title='Weight Distribution')

# 更新坐标轴标题和使用明亮的主题
fig_weight.update_xaxes(title='Weight')
fig_weight.update_layout(template='plotly_white')

# 添加动画效果
fig_weight.update_traces(marker_color='darkblue', opacity=0.7)
fig_weight.update_layout(transition_duration=500)  # 设置动画持续时间

fig_weight.show()

In [27]:
# 创建直方图统计不同位置的球员数量
fig_position = px.histogram(players_data, x='position', title='Player Position Distribution')

# 更新坐标轴标题和使用明亮的主题
fig_position.update_xaxes(title='Position')
fig_position.update_layout(template='plotly_white')

# 添加动画效果
fig_position.update_traces(marker_color='darkblue', opacity=0.7)
fig_position.update_layout(transition_duration=500)  # 设置动画持续时间

fig_position.show()

这张图显示了不同位置的NFL球员数量的分布。从图中我们可以看出以下几点：

- 最常见的位置是WR（外接手），其次是CB（角卫）。
- 相对较少的位置是LS（长开球手），FB（全卫）和MLB（中线卫）。
- 其他位置的球员数量在这两个极端之间，例如QB（四分卫）、T（战术）、TE（近端锋）、DE（防守端锋）等。
- 强卫和游卫/自由位（SS和FS）的球员数量相似，中等偏低。
- 线卫和防守锋线（如OLB、DT、ILB）的球员数量也相近，居中偏上。

总体来说，这张图表反映了各个位置球员的普遍性和在队伍中的分布情况，可能暗示了这些位置在足球战术中的重要性和普遍性。此外，它也可能反映了某些位置对于体能和技能要求的不同，从而影响了人选的广泛性。

In [28]:
# 创建箱线图得到位置与体重的关系，并按中位数排序
fig_box = px.box(players_data, x='position', y='height', title='Position vs Height',
                 category_orders={"position": sorted(players_data['position'].unique(), key=lambda x: -players_data[players_data['position'] == x]['height'].median())})
fig_box.update_xaxes(title='Position')
fig_box.update_yaxes(title='Height')
fig_box.show()

这三个位置的球员身高高
1. **T** - Offensive Tackle（截锋）
2. **TE** - Tight End（边锋）
3. **DE** - Defensive End（防守端锋）

这三个位置的球员身高矮
1. **RB** - Running Back（跑锋）
2. **FB** - Fullback（全卫）
3. **CB** - Cornerback（角卫）

**身高较高的位置：**

1. **Offensive Tackle (T)** - Offensive Tackles通常身高较高，因为他们位于进攻线的两端，负责保护四分卫免受防守球员的压力。较高的身高有助于他们在阻挡防守球员时覆盖更多空间，保护四分卫免受传球时的威胁。

2. **Tight End (TE)** - Tight Ends通常也需要一定的身高，因为他们在进攻中扮演多种角色，既需要接球又需要参与进攻线的阻挡。身高的优势有助于他们在比赛中抓住高空传球，并与防守球员竞争。

3. **Defensive End (DE)** - Defensive Ends通常需要具备一定的身高，以便在防守端扰乱对方的传球和跑球进攻。较高的身高有助于他们在比赛中挡住传球或投掷的视线，并尝试封锁传球。

**身高较矮的位置：**

1. **Running Back (RB)** - Running Backs通常身高较矮，因为他们需要具备速度、敏捷性和低重心，以在球场上穿越防守队员和开辟跑道。较矮的身高使他们更容易躲避对手的防守。

2. **Fullback (FB)** - Fullbacks通常也较矮，他们通常充当跑锋的助手，用于开辟道路并进行阻挡。他们需要在短距离内具备强大的推进力，因此低重心和较矮的身高可以帮助他们更好地履行职责。

3. **Cornerback (CB)** - 角卫通常身高较矮，因为他们需要在比赛中与宽接球员竞争，后者通常身材高大。较矮的身高可以帮助角卫更好地与对方宽接球员对抗，并在比赛中更灵活地移动。

总之，NFL中球员的身高差异是为了适应他们在比赛中的不同角色、职责和体能要求。不同位置需要不同的体格和技能，因此身高差异是合理的。

In [29]:
# 创建箱线图得到位置与体重的关系，并按中位数排序
fig_box = px.box(players_data, x='position', y='weight', title='Position vs Weight',
                 category_orders={"position": sorted(players_data['position'].unique(), key=lambda x: -players_data[players_data['position'] == x]['weight'].median())})
fig_box.update_xaxes(title='Position')
fig_box.update_yaxes(title='Weight')
fig_box.show()

这三个位置的球员体重大
1. **T** - Offensive Tackle（截锋）
2. **NT** - Nose Tackle（正截锋）
3. **G** - Guard（哨锋）

这三个位置的球员体重小
1. **CB** - Cornerback（角卫）
2. **WR** - Wide Receiver（外接手）
3. **FS** - Free Safety（游卫／自由卫）

1. **Offensive Tackle (T)** 和 **Guard (G)**：
   - 进攻线球员（包括左外线、右外线和内线球员）通常需要具备较大的体重和力量，以便在阻挡和保护四分卫时抵挡防守球员的进攻。
   - 他们的任务包括抵挡对方的防守线和创造空间，使进攻得分。体重大有助于他们在肉搏中占据有利位置。
2. **Nose Tackle (NT)**：
   - 正截锋是防守线的一部分，通常需要面对对方的中锋和内线球员。他们的任务包括堵塞跑路和产生对四分卫的压力。
   - 由于需要与对方的内线球员搏斗，正截锋通常具备较大的体重和力量。
3. **Cornerback (CB)**、**Wide Receiver (WR)** 和 **Free Safety (FS)**：
   - 角卫、外接手和游卫/自由位通常需要更多的速度、敏捷性和灵活性，以便在球场上奔跑、转向和防守。
   - 他们的职责涉及覆盖对方的接球手、接收传球、防守长传等，因此对速度和敏捷性要求较高。
   - 较小的体重有助于他们更快地移动和变向。


由此可见，当球员位置处于擒抱与进攻位位时往往需要更强的对抗属性，他们的任务包括抵挡对方的防守线和创造空间，使进攻得分。体重大有助于他们在肉搏中占据有利位置。

当球员为角卫、外接手和游卫/自由位位置时往往需要更高的速度、敏捷性和灵活性，小体重代表了身体灵活敏捷，身体敏捷。他们的职责涉及覆盖对方的接球手、接收传球、防守长传等，较小的体重有助于他们更快地移动和变向。

In [30]:
# 计算BMI指数
players_data['bmi'] = players_data['weight'] / ((players_data['height'] / 100) ** 2)

# 创建箱线图得到位置与体重的关系，并按中位数排序
fig_box = px.box(players_data, x='position', y='bmi', title='Position vs bmi',
                 category_orders={"position": sorted(players_data['position'].unique(), key=lambda x: -players_data[players_data['position'] == x]['bmi'].median())})
fig_box.update_xaxes(title='Position')
fig_box.update_yaxes(title='Bmi')
fig_box.show()

可以看到，BMI指数和球员的体重分布基本一致。

3. plays.csv

首先对该表格内容进行数据清洗与可视化

In [31]:
plays_data.head()

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,...,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,...,0.976785,0.023215,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN
1,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,...,0.160485,0.839515,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,...,0.756661,0.243339,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN
3,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,...,0.620552,0.379448,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN
4,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,...,0.836290,0.163710,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN


In [32]:
# 调用函数并打印生成的表格
plays_data_summary = generate_summary_table(plays_data)

表格信息统计:


,Column,Data Type,Missing Values,Missing %,Unique Values,Min,Max,Mean,Median
0,gameId,int64,0,0.00%,136,2022090800,2022110700,2022098953.855598,2022100903.000000
1,playId,int64,0,0.00%,3974,54,5096,1986.603476,1990.500000
2,ballCarrierId,int64,0,0.00%,480,25511,55158,48072.271664,47789.000000
3,ballCarrierDisplayName,object,0,0.00%,480,,,,
4,playDescription,object,0,0.00%,12486,,,,
5,quarter,int64,0,0.00%,5,1,5,2.550136,3.000000
6,down,int64,0,0.00%,4,1,4,1.727054,2.000000
7,yardsToGo,int64,0,0.00%,32,1,38,8.469085,10.000000
8,possessionTeam,object,0,0.00%,32,,,,
9,defensiveTeam,object,0,0.00%,32,,,,


不同节次中比赛的分布情况

In [33]:
# 创建直方图
fig = px.histogram(plays_data, x="quarter", title="Distribution of Games by Quarters")

# 修改颜色方案并设置opacity
color_sequence = px.colors.qualitative.Plotly
fig.update_traces(marker=dict(color='darkblue', opacity=0.7))  # 设置颜色和opacity

# 添加渐变填充
fig.update_traces(marker=dict(line=dict(color='white', width=2)))

# 使用自定义字体和标题样式
fig.update_layout(font=dict(family="Arial", size=12, color="black"))
fig.update_layout(title_font=dict(family="Times New Roman", size=24, color="navy"))

# 修改布局
fig.update_layout(
    xaxis_title="Quarter",
    yaxis_title="Number of Games",
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    margin=dict(l=80, r=80, t=80, b=80),
)

fig.show()

这张图展示了按季度划分的游戏数量分布。通常，一个NFL比赛包括4个季度，但是图中还包括了第5季度，这可能表示加时赛。根据图表，我们可以得出以下结论：
* 第1、2和4节的数量相对较高，这可能表示大多数比赛在常规时间内结束。
* 第3节的数量略低，这可能是由于在半场休息后，某些比赛的持球时间变长或许多比赛在第三节结束前就已经决定了胜负。
* 第5节的数量显著减少，这表明只有少数比赛进入加时赛。
整体上，此图反映了大多数NFL比赛在常规四节内决出胜负，而只有少数比赛需要进行加时赛。

进攻次数在比赛的分布情况

In [34]:
# 获取不同down的唯一整数值
down_values = sorted(plays_data['down'].unique())

# 创建直方图
fig = px.histogram(plays_data, x='down', title='Distribution of Plays by Down',
                   labels={'down': 'Down', 'count': 'Number of Plays'})

# 自定义横坐标轴的刻度值和标签
fig.update_layout(xaxis=dict(tickvals=down_values, ticktext=[str(val) for val in down_values]))

# 将柱子颜色改成碧蓝色，并设置透明度为0.7
fig.update_traces(marker_color='darkblue', opacity=0.7)

# 使用自定义字体和标题样式
fig.update_layout(font=dict(family="Arial", size=12, color="black"))
fig.update_layout(title_font=dict(family="Times New Roman", size=24, color="navy"))

# 添加阴影效果
fig.update_traces(marker=dict(line=dict(width=2, color='white')))

# 显示图形
fig.show()

这张图展示了NFL比赛中按攻防回合划分的进攻机会次数的分布。持球的一队（进攻方）有四次进攻机会向前（防守方的端区）推进10码，每次机会称为一个“档”，（down，即被对方拦截放倒一次）根据图表，可以得出以下结论：

- 第1档有最多的比赛次数，这是因为每个新的进攻都从第1回合开始。
- 第2档的次数减少，但仍然相对较高，这表明许多进攻在第1档后仍在继续。
- 到了第3档，次数进一步减少，这可能反映了在前两个档没有获得足够的推进码数
- 第4档的次数最少，这是因为许多队伍在第3档后采用射门（Field goal）或弃踢（punt）将球权转移给对手但令他们必须从较远的地方开始进攻。

总体上，这张图反映了在NFL比赛中，随着进攻回合的增加，进行比赛的次数递减，这可能与球队的战略选择有关，以及在进攻回合中获得必要进程以保持球权的难度增加。

前进码数（yards to go）的分布情况

In [35]:
# 获取不同yardsToGo的唯一整数值
yardsToGo_values = sorted(plays_data['yardsToGo'].unique())

# 创建直方图
fig = px.histogram(plays_data, x='yardsToGo', title='Distribution of Plays by yards to go',
                   labels={'yardsToGo': 'Yards to Go', 'count': 'Number of Plays'})

# 自定义横坐标轴的刻度值和标签
fig.update_layout(xaxis=dict(tickvals=yardsToGo_values, ticktext=[str(val) for val in yardsToGo_values]))

# 将柱子颜色改成碧蓝色，并设置透明度为0.7
fig.update_traces(marker_color='darkblue', opacity=0.7)

# 使用自定义字体和标题样式
fig.update_layout(font=dict(family="Arial", size=12, color="black"))
fig.update_layout(title_font=dict(family="Times New Roman", size=24, color="navy"))

# 添加阴影效果
fig.update_traces(marker=dict(line=dict(width=2, color='white')))

# 显示图形
fig.show()


这张图表展示了NFL比赛中根据需要前进的码数（即距离新一组四次进攻开始位置的码数）来划分的比赛次数分布。分析图表，我们可以看出：

- 最常见的情况是仅需前进10码，这通常是在一组新的四次进攻开始时的标准距离。
- 当需要前进的码数较少时，比如1到4码，比赛次数相对较少，这可能是由于成功获得了新的一组四次进攻或得分的比赛次数较多。
- 需要前进的码数超过10码时，比赛次数急剧下降，这可能是由于犯规判罚导致球队处于不利位置的情况较少发生。

总的来说，这张图表表明大部分比赛的进攻都是从标准的10码开始的，而需要超过10码才能获得新一组四次进攻的情况相对较少。

进攻阵型的分布情况

In [36]:
# 获取不同offenseFormation的频数并按照频数大小排序
offenseFormation_counts = plays_data['offenseFormation'].value_counts().reset_index()
offenseFormation_counts.columns = ['offenseFormation', 'count']
offenseFormation_counts = offenseFormation_counts.sort_values(by='count', ascending=False)

# 创建直方图
fig = px.bar(offenseFormation_counts, x='offenseFormation', y='count', 
             title='Distribution of Plays by offense formation',
             labels={'offenseFormation': 'Offense Formation', 'count': 'Number of Plays'})

# 自定义横坐标轴的刻度值和标签
fig.update_layout(xaxis=dict(tickvals=offenseFormation_counts['offenseFormation'], 
                             ticktext=offenseFormation_counts['offenseFormation']))

# 将柱子颜色改成碧蓝色，并设置透明度为0.7
fig.update_traces(marker_color='darkblue', opacity=0.7)

# 使用自定义字体和标题样式
fig.update_layout(font=dict(family="Arial", size=12, color="black"))
fig.update_layout(title_font=dict(family="Times New Roman", size=24, color="navy"))

# 添加阴影效果
fig.update_traces(marker=dict(line=dict(width=2, color='white')))

# 显示图形
fig.show()

图表显示了NFL比赛中不同进攻阵型所执行的比赛次数分布。根据图表，我们可以得出以下结论：

- "SHOTGUN"（霰弹枪阵式）阵型是最常使用的进攻阵型，执行的比赛次数最多。
- "SINGLEBACK"（单后卫）阵型是第二常用的阵型。
- "I_FORM"（I型）和"Pistol"（手枪）阵型的使用频率相对较低。
- "EMPTY"（空背场）、"JUMBO"（巨型）和"WILDCAT"（野猫）等阵型的使用次数较少，可能是特定情况下的战术选择。
- 图表最后一个分类为"nan"，这通常表示数据中的缺失值或未分类的阵型。

这张图表反映了各种进攻阵型在实际比赛中的普及度。"SHOTGUN"阵型的高频使用可能是因为它适用于多种比赛情况，而更专门的阵型如"JUMBO"和"WILDCAT"可能仅在特定战术或比赛情况下使用。这种信息对于分析团队的战术偏好和比赛风格特别有价值。

4. tackles.csv

首先对该表格内容进行数据清洗与可视化

In [37]:
tackles_data.head()

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800,101,42816,1,0,0,0
1,2022090800,393,46232,1,0,0,0
2,2022090800,486,40166,1,0,0,0
3,2022090800,646,47939,1,0,0,0
4,2022090800,818,40107,1,0,0,0


In [38]:
# 调用函数并打印生成的表格
tackles_data_summary = generate_summary_table(tackles_data)

表格信息统计:


,Column,Data Type,Missing Values,Missing %,Unique Values,Min,Max,Mean,Median
0,gameId,int64,0,0.00%,136,2022090800,2022110700,2022098971.441123,2022100903.000000
1,playId,int64,0,0.00%,3943,54,5096,1982.974578,1991.000000
2,nflId,int64,0,0.00%,800,33131,55241,47602.719442,46669.000000
3,tackle,int64,0,0.00%,2,0,1,0.569207,1.000000
4,assist,int64,0,0.00%,2,0,1,0.315276,0.000000
5,forcedFumble,int64,0,0.00%,2,0,1,0.005681,0.000000
6,pff_missedTackle,int64,0,0.00%,2,0,1,0.119936,0.000000


可以看到该表全为数值项，并且没有任何缺失数据，大大减轻了笔者的工作！！！
直接进行可视化分析！！！

铲球（tackles）的分布情况

In [39]:
# 统计铲球是否成功的次数
tackle_success_counts = tackles_data['tackle'].value_counts()

# 创建条形统计图
fig = px.bar(x=tackle_success_counts.index, y=tackle_success_counts.values,
             labels={'x': 'Tackle Success', 'y': 'Count'},
             title='Distribution of Tackle Success',
             category_orders={'x': [0, 1]}
            )

# 设置 x 轴标签
fig.update_xaxes(type='category')

# 在每个条形上方添加文本标签
for i, count in enumerate(tackle_success_counts.values):
    fig.add_annotation(
        x=tackle_success_counts.index[i],
        y=count,
        text=str(count),
        showarrow=True,
        font=dict(size=12)
    )

# 将柱子颜色改成碧蓝色，并设置透明度为0.7
fig.update_traces(marker_color='darkblue', opacity=0.7)

# 使用自定义字体和标题样式
fig.update_layout(font=dict(family="Arial", size=12, color="black"))
fig.update_layout(title_font=dict(family="Times New Roman", size=24, color="navy"))

# 添加阴影效果
fig.update_traces(marker=dict(line=dict(width=2, color='white')))

# 显示图表
fig.show()

该图展示了球员进行铲球动作并且铲球成功的条形统计图。

值得注意的是，当值为0时，代表的是球员并没有执行铲球动作或者执行了铲球动作但是铲球失败的情况。

助攻（assists）的分布情况

In [40]:
# 统计助攻是否成功的次数
assist_counts = tackles_data['assist'].value_counts()

# 创建条形统计图
fig = px.bar(x=assist_counts.index, y=assist_counts.values,
             labels={'x': 'Assist Success', 'y': 'Count'},
             title='Distribution of Assist Success',
             category_orders={'x': [0, 1]}
            )

# 设置 x 轴标签
fig.update_xaxes(type='category')

# 在每个条形上方添加文本标签
for i, count in enumerate(assist_counts.values):
    fig.add_annotation(
        x=assist_counts.index[i],
        y=count,
        text=str(count),
        showarrow=True,
        font=dict(size=12)
    )

# 将柱子颜色改成碧蓝色，并设置透明度为0.7
fig.update_traces(marker_color='darkblue', opacity=0.7)

# 使用自定义字体和标题样式
fig.update_layout(font=dict(family="Arial", size=12, color="black"))
fig.update_layout(title_font=dict(family="Times New Roman", size=24, color="navy"))

# 添加阴影效果
fig.update_traces(marker=dict(line=dict(width=2, color='white')))

# 显示图表
fig.show()

该图展示了球员进行助攻动作并且助攻铲球成功的条形统计图。

值得注意的是，当值为0时，代表的是球员并没有执行助攻动作或者执行了助攻动作但是助攻失败的情况。

铲球失败的分布情况

In [41]:
# 统计铲球失败的次数
missed_tackles_counts = tackles_data['pff_missedTackle'].value_counts()

# 创建条形统计图
fig = px.bar(x=missed_tackles_counts.index, y=missed_tackles_counts.values,
             labels={'x': 'PFF Missed Tackle', 'y': 'Count'},
             title='Distribution of PFF Missed Tackles',
             category_orders={'x': [0, 1]}
            )

# 设置 x 轴标签
fig.update_xaxes(type='category')

# 在每个条形上方添加文本标签
for i, count in enumerate(missed_tackles_counts.values):
    fig.add_annotation(
        x=missed_tackles_counts.index[i],
        y=count,
        text=str(count),
        showarrow=True,
        font=dict(size=12)
    )

# 将柱子颜色改成碧蓝色，并设置透明度为0.7
fig.update_traces(marker_color='darkblue', opacity=0.7)

# 使用自定义字体和标题样式
fig.update_layout(font=dict(family="Arial", size=12, color="black"))
fig.update_layout(title_font=dict(family="Times New Roman", size=24, color="navy"))

# 添加阴影效果
fig.update_traces(marker=dict(line=dict(width=2, color='white')))

# 显示图表
fig.show()

该图展示了球员铲球失败的条形统计图。

值得注意的是，当值为0时，代表的是球员铲球成功和助攻成功的总和，只有值为1时才是绝对的铲球失误。

* 除此之外，表格中还出现了pff_missedTackle和tackle的值同时为1的情况出现，对于这样的情况是数据统计的标准选取不同造成的，在一段gameplay当中，铲球球员可能会进行多次铲球动作，当头几次失败，之后又成功时，这样的统计量就会出现，造成pff_missedTackle和tackle的值同时为1